In [ ]:
import cv2
import numpy as np

from glob import glob
from random import randint
from scipy.optimize import linear_sum_assignment

In [ ]:
paths = sorted(glob('./CS585-BatImages/Gray/*'))

In [ ]:
def getCont(path):
    img = cv2.imread(path, 0)
    img = cv2.blur(img, (5, 5))
    
    a_thres_output = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, -5)
    contours, _ = cv2.findContours(a_thres_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

def getCent(c):
    points = [i[0] for i in c]
    x, y = zip(*points)
    x, y = np.mean(x), np.mean(y)
    return (x, y)

det = lambda x: [getCent(i) for i in getCont(x)]

In [ ]:
# dets = [det(i) for i in paths]
def readFile(fp):
    with open(fp, 'r') as file:
        data = file.read().strip()
    
    data = data.split('\n')
    f = lambda x: np.array(list(map(int, x.split(','))), np.float32)
    return list(map(f, data))



detp = sorted(glob('./CS585-Bats-Localization/Localization/*'))
dets = [readFile(i) for i in detp]


In [ ]:
class Obj:
    def __init__(self, start_measure):
        self.filter = cv2.KalmanFilter(4, 2)
        self.color = (randint(80, 255), randint(80, 255), randint(80, 255))

        
        self.filter.measurementMatrix = np.array([[1, 0, 0, 0],
                                                  [0, 1, 0, 0]], np.float32)

        self.filter.transitionMatrix = np.array([[1, 0, 1, 0],
                                                 [0, 1, 0, 1],
                                                 [0, 0, 1, 0],
                                                 [0, 0, 0, 1]], np.float32)
         
        
        for i in range(10):
            self.filter.predict()
            self.filter.correct(start_measure)
        
        
        self.history = []
        self.tracks = 0
        
        self._miss = 0
        
    def getPred(self):
            pred =  self.filter.predict()
            pred = np.array([pred[0][0], pred[1][0]])

            self.history.append(pred)
            self.tracks += 1
        
            return pred
    
    def update(self, measure):
        self.filter.correct(measure)
        self._miss = 0
        
    def miss(self):
        self._miss += 1
        if self._miss >= 6:
            return True
        else:
#             measure = self.getPred()
#             self.filter.correct(measure)
        
#             self.history.append(measure)
#             self.tracks += 1
            
            return False
        
    def draw(self, img):
        if self.tracks>=3:
            pts = np.array([[np.rint(i).astype('int32').tolist() for i in self.history[2:]]])
#             print(pts[-1,-1,:])
            cv2.polylines(img, pts, False, self.color, 1, lineType=cv2.LINE_8)
        return img    

In [ ]:
sq = lambda x: x*x
dist = lambda x, y: sum(sq(x-y))**0.5

def getCM(est, meas):
    x, y = len(est), len(meas)
    
    cm = 25*np.ones((x+y, x+y), dtype = np.float64)
    for i in range(x):
        for j in range(y):
            cm[i, x+j] = cm[x+j, i] = dist(est[i], meas[j])
    return cm

In [ ]:
objs =  [Obj(i) for i in dets[0]]


# fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video=cv2.VideoWriter('video.avi', fourcc, 1,(1024, 1024))


for ind, det in enumerate(dets[1:]):
    CM = getCM([obj.getPred() for obj in objs], det)
    obj_v = [False for i in objs]
    mea_v = [False for i in det]
    
    row_ind, col_ind = linear_sum_assignment(CM)
    
    prop = (row_ind!=col_ind)
    row_ind, col_ind = row_ind[prop], col_ind[prop]
    
    mi = (row_ind<len(objs)).sum()
    row_ind, col_ind = row_ind[:mi], col_ind[:mi]
    
    col_ind -= len(objs)
    prop = (col_ind>=0)
    row_ind, col_ind = row_ind[prop], col_ind[prop]
    
#     print(row_ind, len(objs))
#     print(col_ind, len(det))
    for i in range(row_ind.shape[0]):
        objs[row_ind[i]].update(det[col_ind[i]])
        obj_v[row_ind[i]], mea_v[col_ind[i]] = True, True
    
    miss = [i for i in [i for i, j in enumerate(obj_v) if not j] if objs[i].miss()]
    miss = [j-i for i, j in enumerate(miss)]
    for i in miss:
        objs.pop(i)
         
    
    for i in [i for i, j in enumerate(mea_v) if not j]:
        objs.append(Obj(det[i]))
    
    img = cv2.imread(paths[ind+1], 0)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for obj in objs:
        img = obj.draw(img)
    
    cv2.imshow('tracking', img)
#     video.write(img)
    cv2.waitKey(0);

cv2.destroyAllWindows()
# video.release()